# Neural Network Regression

## Import useful librairies/fonctions

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch as nn
import math as m
import time
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(10,10)})

## Import data

### Fonctions

In [ ]:
def select_data(instance, num, path=None):
    #Read
    if path == None :
        df = pd.read_csv(f"HGS/{instance}/XML100_{instance}_{num}.csv", sep=';')
    else :
        df = pd.read_csv(path, sep=';')
        path = path.split("_")
        instance = path[-2]
        num = path[-1].replace(".csv", "")

    #Parse the row which was parsed as columns
    new_row = {}
    for ele in df.columns:
        new_row[ele] = ele
    df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

    #Change Columns Names
    df.columns = ["name", "cost","S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17", "S18"]
    
    #Change type of columns
    df["cost"] = df["cost"].astype(int)
    df["name"] = df["name"].astype(str)
    df = df.drop(columns=["name"])
    for k in range(1,19):
        df[f"S{k}"] = df[f"S{k}"].astype(float)
    
    df["instance"] = int(instance)
    df["num"] = int(num)

    return df

def fusion_intra_instance(instance):
    df_tot= select_data(0,0, f"HGS//{instance}//"+os.listdir(f"HGS//{instance}")[0])
    first = True

    for ele in os.listdir(f"HGS//{instance}"):

        if first:
            first = False
        else : 
            df = select_data(0,0, f"HGS//{instance}//"+ele)
            df_tot = pd.concat([df_tot, df], ignore_index=True)

    return df_tot


def fusion_inter_instance():
    df_tot = fusion_intra_instance("2113")
    for instance in ['2213', "3113", "3213"]:
        df = fusion_intra_instance(instance)
        df_tot = pd.concat([df_tot, df], ignore_index=True)
    return df_tot

def reduce_size(df_tot):
    for col in df_tot.columns :
        if col not in ["cost", "instance", "num"]:
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='float')
        else : 
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='unsigned')
    return df_tot

In [ ]:
def import_dataset(path):
    df = pd.read_csv(path, sep=';')
    df.columns = ["Name", "Cost","S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17", "S18"]
    return df

def reduce_size(df_tot):
    for col in df_tot.columns :
        if col not in ["Configuration", "Instance", "Solution", "Cost"]:
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='float')
        else : 
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='integer')
    return df_tot

def tirage_repartition(indice, nb):
    tirage = []
    bond = int((len(indice)-1)/(nb-1))
    for k in range(0, len(indice), bond):
        tirage.append(indice[k])
    
    for element in tirage:
        indice.remove(element)

    return tirage, indice
    
def create_samples(p_train, nb_train, nb_test, nb_verify, seed):
    # Mettre la même seed pour avoir des résultats constants
    np.random.seed(seed)

    # Création des variables utiles
    fichiers_2113 = os.listdir('../HGS/2113/')
    fichiers_2213 = os.listdir('../HGS/2213/')
    fichiers_3113 = os.listdir('../HGS/3113/')
    fichiers_3213 = os.listdir('../HGS/3213/')

    #Shuffle 
    np.random.shuffle(fichiers_2113)
    np.random.shuffle(fichiers_2213)
    np.random.shuffle(fichiers_3113)
    np.random.shuffle(fichiers_3213)

    #Choix des fichiers trains et test (On arrondie au supérieur)
    train_list = {}
    test_list = {}
    train_list["2113"], test_list["2113"] = fichiers_2113[:m.ceil(p_train*len(fichiers_2113))], fichiers_2113[m.ceil(p_train*len(fichiers_2113)):]
    train_list["2213"], test_list["2213"] = fichiers_2213[:m.ceil(p_train*len(fichiers_2213))], fichiers_2213[m.ceil(p_train*len(fichiers_2213)):]
    train_list["3113"], test_list["3113"] = fichiers_3113[:m.ceil(p_train*len(fichiers_3113))], fichiers_3113[m.ceil(p_train*len(fichiers_3113)):]
    train_list["3213"], test_list["3213"] = fichiers_3213[:m.ceil(p_train*len(fichiers_3213))], fichiers_3213[m.ceil(p_train*len(fichiers_3213)):]

    #On créer les trois data_frames
    colonnes = ["Configuration", "Instance", "Solution", "Cost","S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17", "S18"]
    lignes_train = []
    lignes_verify = []
    lignes_test = []


    for instance in train_list.keys():
        fichiers_train = train_list[instance]
        fichiers_test = test_list[instance]

        for path_fichier in fichiers_train:
            df = import_dataset(f"../HGS/{instance}/" + path_fichier)
            path_fichier = path_fichier.split("_")
            num = path_fichier[-1].replace(".csv", "")

            #On trie le dataset pour avoir un échantillon représentatif 
            df = df.sort_values(by = "Cost", ascending=False)
            df = df.reset_index().drop(columns=["index"])


            indice = [k for k in range(df.shape[0])]
            tirage_train, indice = tirage_repartition(indice, nb_train)
            tirage_verify, indice = tirage_repartition(indice, nb_verify)

            for i in range(nb_train):
                row_index = tirage_train.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_train.append(row)
                
            for i in range(nb_verify):
                row_index = tirage_verify.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_verify.append(row)
                

        for path_fichier in fichiers_test:
            df = import_dataset(f"../HGS/{instance}/" + path_fichier)
            path_fichier = path_fichier.split("_")
            num = path_fichier[-1].replace(".csv", "")

            #On trie le dataset pour avoir un échantillon représentatif 
            df = df.sort_values(by = "Cost", ascending=False)
            df = df.reset_index().drop(columns=["index"])
            indice = [k for k in range(df.shape[0])]
            tirage_test, indice = tirage_repartition(indice, nb_test)
            tirage_verify, indice = tirage_repartition(indice, nb_verify)

            for i in range(nb_test):
                row_index = tirage_test.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_test.append(row)
            for i in range(nb_verify):
                row_index = tirage_verify.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_verify.append(row)
        


    #Création des data frames

    df_train = pd.DataFrame(lignes_train, columns=colonnes)
    df_verify = pd.DataFrame(lignes_verify, columns=colonnes)
    df_test = pd.DataFrame(lignes_test, columns=colonnes)

    df_train = df_train.drop(columns=['S7'])
    df_verify = df_verify.drop(columns=['S7'])
    df_test = df_test.drop(columns=['S7'])
    return reduce_size(df_train), reduce_size(df_verify), reduce_size(df_test)


### Data

In [ ]:
df_train, df_verify, df_test = create_samples(0.8, 10000, 1000, 1000, 1)

In [ ]:
print("df_train shape", df_train.shape)
print("df_verify shape", df_verify.shape)
print("df_test shape", df_test.shape)

In [ ]:
X_train, Y_train = df_train.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_train['Cost']
X_verify, Y_verify = df_verify.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_verify['Cost']
X_test, Y_test = df_test.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_test['Cost']


## Neural Network

### Neural network architecture

In [ ]:
class MLP(nn.nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.nn.Sequential(
      nn.nn.Linear(17, 26),
      nn.nn.ReLU(),
      nn.nn.Linear(26, 19),
      nn.nn.ReLU(),
      nn.nn.Linear(19, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

### Instanciating the model

#### Device setup

In [ ]:
# Check if GPU is available
is_cuda = nn.cuda.is_available()

# Select it as default, or CPU otherwise
print(is_cuda)
if is_cuda:
    device = nn.device("cuda")
else:
    device = nn.device("cpu")

#### Scaling Data

In [ ]:
X_train_f = StandardScaler().fit(X_train).transform(X_train)
Y_train_f = StandardScaler().fit(Y_train.values.reshape(-1,1)).transform(Y_train.values.reshape(-1,1))

X_verify_f = StandardScaler().fit(X_verify).transform(X_verify)
Y_verify_f = StandardScaler().fit(Y_verify.values.reshape(-1,1)).transform(Y_verify.values.reshape(-1,1))

X_test_f = StandardScaler().fit(X_test).transform(X_test)
Y_test_f = StandardScaler().fit(Y_test.values.reshape(-1,1)).transform(Y_test.values.reshape(-1,1))

print("Done")

#### Converting to tensor

In [ ]:
from torch.autograd import Variable 

X_train_t = Variable(nn.cuda.FloatTensor(X_train_f)).to(device)
Y_train_t = Variable(nn.cuda.FloatTensor(Y_train_f)).to(device)

X_test_t = Variable(nn.cuda.FloatTensor(X_test_f)).to(device)
Y_test_t = Variable(nn.cuda.FloatTensor(Y_test_f)).to(device)

X_verify_t = Variable(nn.cuda.FloatTensor(X_verify_f)).to(device)
Y_verify_t = Variable(nn.cuda.FloatTensor(Y_verify_f)).to(device)

print("Done")

#### Databatch setup

In [ ]:
databatch_train = nn.utils.data.TensorDataset(X_train_t, Y_train_t)
databatch_test = nn.utils.data.TensorDataset(X_test_t, Y_test_t)
databatch_verify = nn.utils.data.TensorDataset(X_verify_t, Y_verify_t)

trainloader_train = nn.utils.data.DataLoader(databatch_train, batch_size=10, shuffle=True, num_workers=0)
trainloader_test = nn.utils.data.DataLoader(databatch_test, batch_size=10, shuffle=True, num_workers=0)
trainloader_verify = nn.utils.data.DataLoader(databatch_verify, batch_size=10, shuffle=True, num_workers=0)

#### Model, Loss function and optimizer

In [ ]:
from torchmetrics import MeanAbsolutePercentageError

mean_abs_percentage_error = MeanAbsolutePercentageError().to(device)


In [ ]:
# Initialize the MLP
mlp = MLP().to(device)

# Define the loss function and optimizer
loss_function = nn.nn.MSELoss()
optimizer = nn.optim.Adam(mlp.parameters(), lr=1e-4)

#### Training

In [ ]:
start = time.time()

train_losses, test_losses, verify_losses, mape_losses = [], [], [], []
n_epochs = 20 #Overfit
# Run the training loop
for epoch in range(0, n_epochs): # 10 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    train_loss, test_loss, verify_loss, mape_loss = 0.0, 0.0, 0.0, 0.0

    # Iterate over the DataLoader for training data
    for data_train, data_test, data_verify  in zip(trainloader_train, trainloader_test, trainloader_verify):
        
        # Get and prepare inputs
        inputs, targets =  data_train
        inputs_test, targets_test =  data_test
        inputs_verify, targets_verify =  data_verify
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = mlp(inputs)
        
        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Loss
        train_loss += loss.item()
        
        with nn.no_grad():
            mlp.eval()
            pred_ytest = mlp.forward(inputs_test)
            test_loss += loss_function(pred_ytest, targets_test).item()

        with nn.no_grad():
            mlp.eval()
            pred_ytest = mlp.forward(inputs_verify)
            verify_loss += loss_function(pred_ytest, targets_verify).item()
        
        with nn.no_grad():
            mlp.eval()
            pred_ytest = mlp.forward(inputs_test)
            mape_loss += mean_abs_percentage_error(pred_ytest, targets_verify).item()

    train_losses.append(train_loss / len(trainloader_train))
    test_losses.append(test_loss / len(trainloader_test))
    verify_losses.append(verify_loss / len(trainloader_verify))
    mape_losses.append(mape_loss / len(trainloader_test))
        
# Process is complete.
print('Training process has finished.')
end = time.time()
elapsed = end - start
print("Temps exécution : ", elapsed)

#### Plot training 

In [ ]:
plt.plot(
    np.array(train_losses).reshape((n_epochs, -1)).mean(axis=1),
    label='Training loss'
)
plt.plot(
    np.array(test_losses).reshape((n_epochs, -1)).mean(axis=1),
    label='Test loss'
)

plt.plot(
    np.array(verify_losses).reshape((n_epochs, -1)).mean(axis=1),
    label='Verify loss'
)


plt.legend(frameon=False)
plt.xlabel('epochs')
plt.ylabel('L1Loss')

In [ ]:
plt.plot(
    np.array(mape_losses).reshape((n_epochs, -1)).mean(axis=1),
    label='Training loss'
)

plt.legend(frameon=False)
plt.xlabel('epochs')
plt.ylabel('MAPE Loss')

## Tuning some shit

In [ ]:
class MLP_tuning(nn.nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self, x ,y,z):
    super().__init__()
    self.layers = nn.nn.Sequential(
      nn.nn.Linear(17, x),
      nn.nn.ReLU(),
      nn.nn.Linear(x, y),
      nn.nn.ReLU(),
      nn.nn.Linear(y, z),
      nn.nn.ReLU(),
      nn.nn.Linear(z, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [ ]:
def training(x,y,z, lr):
    # Initialize the MLP
    mlp = MLP_tuning(x,y,z).to(device)

    # Define the loss function and optimizer
    loss_function = nn.nn.L1Loss()
    optimizer = nn.optim.Adam(mlp.parameters(), lr=lr)

    train_losses, test_losses, verify_losses = [], [], []
    n_epochs = 10 #Overfit
    # Run the training loop
    for epoch in range(0, n_epochs): # 10 epochs at maximum

        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        train_loss, test_loss, verify_loss = 0.0, 0.0, 0.0

        # Iterate over the DataLoader for training data
        for data_train, data_test, data_verify  in zip(trainloader_train, trainloader_test, trainloader_verify):
            
            # Get and prepare inputs
            inputs, targets =  data_train
            inputs_test, targets_test =  data_test
            inputs_verify, targets_verify =  data_verify
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Perform forward pass
            outputs = mlp(inputs)
            
            # Compute loss
            loss = loss_function(outputs, targets)
            
            # Perform backward pass
            loss.backward()
            
            # Perform optimization
            optimizer.step()
            
            # Loss
            train_loss += loss.item()
            
            with nn.no_grad():
                mlp.eval()
                pred_ytest = mlp.forward(inputs_test)
                test_loss += loss_function(pred_ytest, targets_test).item()

            with nn.no_grad():
                mlp.eval()
                pred_ytest = mlp.forward(inputs_verify)
                verify_loss += loss_function(pred_ytest, targets_verify).item()

            train_losses.append(train_loss / len(trainloader_train))
            test_losses.append(test_loss / len(trainloader_test))
            verify_losses.append(verify_loss / len(trainloader_verify))
    # Process is complete.
    print('Training process has finished.')

    return  min(train_losses), min(test_losses), min(verify_losses)
    

In [ ]:
values = {}

for x in range(1,32):
    for y in range(1,32):
        for z in range(1,32):
            for lr in [0.0001, 0.0002, 0.0003]:
                train_loss, test_loss, verify_loss = training(x,y,z,lr)
                values[(x,y,z,lr)] = (train_loss, test_loss, verify_loss)
best_value = 1
best= None
for key, value in values.item():
    if value[1] < best_value:
        best_value=value[1]
        best = key

print(best, best_value)